Proceso ETL del archivo json australian_user_items

In [2]:
import pandas as pd 
import ast

In [3]:
# Lista para almacenar los diccionarios JSON de cada línea
data_list = []

# Ruta del archivo JSON
file_path = 'Datasets/australian_users_items.json'

# Abrir el archivo y procesar cada Línea
with open(file_path, 'r', encoding='utf-8') as file:

    try:
        for line in file:

            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)

            # Agregar el diccionario a la lista
            data_list.append(json_data)

    except ValueError as e:
        print(f"Error en la línea: {line}")
    

In [4]:
data_items = pd.DataFrame(data_list)

In [5]:
# Se desanidan los datos de la columna 'items'
data_1 = data_items.explode(['items'])

# Se aplica la función pd.Series a la columna 'items'
data_2 = pd.json_normalize(data_1['items']).set_index(data_1['items'].index)

# Se concatenan los DataFrames data_1 y data_2
data = pd.concat([data_2, data_1], axis=1)

# Se muestran las primeras 5 filas del DataFrame
data.head()

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id,user_url,items
0,10,Counter-Strike,6.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '10', 'item_name': 'Counter-Strike..."
0,20,Team Fortress Classic,0.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '20', 'item_name': 'Team Fortress ..."
0,30,Day of Defeat,7.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '30', 'item_name': 'Day of Defeat'..."
0,40,Deathmatch Classic,0.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '40', 'item_name': 'Deathmatch Cla..."
0,50,Half-Life: Opposing Force,0.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '50', 'item_name': 'Half-Life: Opp..."


In [6]:
data_d = data.drop(['items','user_url'],axis=1)

In [7]:
data_d

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id
0,10,Counter-Strike,6.0,0.0,76561197970982479,277,76561197970982479
0,20,Team Fortress Classic,0.0,0.0,76561197970982479,277,76561197970982479
0,30,Day of Defeat,7.0,0.0,76561197970982479,277,76561197970982479
0,40,Deathmatch Classic,0.0,0.0,76561197970982479,277,76561197970982479
0,50,Half-Life: Opposing Force,0.0,0.0,76561197970982479,277,76561197970982479
...,...,...,...,...,...,...,...
88308,373330,All Is Dust,0.0,0.0,76561198329548331,7,76561198329548331
88308,388490,One Way To Die: Steam Edition,3.0,3.0,76561198329548331,7,76561198329548331
88308,521570,You Have 10 Seconds 2,4.0,4.0,76561198329548331,7,76561198329548331
88308,519140,Minds Eyes,3.0,3.0,76561198329548331,7,76561198329548331


In [8]:
#Reseteamos el indice del dataframe 
data_items = data_d.reset_index(drop=True) 

In [9]:
#Procedemos a ordenar las columnas del dataframe para verlo de una manera más eficiente
columns = data_items.columns
new_columns_order = ['item_id','item_name','playtime_forever','playtime_2weeks','items_count','user_id','steam_id']
data_items = data_items[new_columns_order]
#Cambio de los nombres de las columnas
new_names_columns = {'item_id':'Id_item','item_name':'Item_name','playtime_forever':'Playtime_forever','playtime_2weeks':'Playtime_2weeks','user_id':'Id_user','items_count':'Items_count','steam_id':'Steam_id'}
data_items.rename(columns=new_names_columns,inplace=True)

In [10]:
#Elimina los valores nulos de las columnas seleccionadas 
data_items= data_items.dropna(subset=['Id_item'])
data_items= data_items.dropna(subset=['Playtime_forever'])
data_items= data_items.dropna(subset=['Playtime_2weeks'])

In [11]:
#Analizamos los datos en cada columna
data_items.describe()

,Playtime_forever,Playtime_2weeks,Items_count
count,5.153209e+06,5.153209e+06,5.153209e+06
mean,9.914951e+02,9.104707e+00,3.147229e+02
std,5.418204e+03,1.403926e+02,6.933596e+02
min,0.000000e+00,0.000000e+00,1.000000e+00
25%,0.000000e+00,0.000000e+00,7.200000e+01
50%,3.400000e+01,0.000000e+00,1.380000e+02
75%,3.550000e+02,0.000000e+00,2.760000e+02
max,6.427730e+05,1.996700e+04,7.762000e+03


In [12]:
#Guarda las filas de la columna Playtime_forever diferente a 0
data_items = data_items[data_items['Playtime_forever']!= 0]

In [13]:
#Carga el dataframe con las columnas necesarias para el analisis
data_items_load = data_items[['Id_item','Item_name','Playtime_forever', 'Id_user',	'Steam_id']]

In [14]:
data_items_load

,Id_item,Item_name,Playtime_forever,Id_user,Steam_id
0,10,Counter-Strike,6.0,76561197970982479,76561197970982479
2,30,Day of Defeat,7.0,76561197970982479,76561197970982479
8,300,Day of Defeat: Source,4733.0,76561197970982479,76561197970982479
9,240,Counter-Strike: Source,1853.0,76561197970982479,76561197970982479
10,3830,Psychonauts,333.0,76561197970982479,76561197970982479
...,...,...,...,...,...
5170007,304930,Unturned,677.0,76561198329548331,76561198329548331
5170008,227940,Heroes & Generals,43.0,76561198329548331,76561198329548331
5170011,388490,One Way To Die: Steam Edition,3.0,76561198329548331,76561198329548331
5170012,521570,You Have 10 Seconds 2,4.0,76561198329548331,76561198329548331


In [15]:
#Exporta a csv 
data_items_load.to_csv('Load_data/CSV/User_items_output.csv', index=False)